In [9]:
import pandas as pd


df = pd.read_csv('./dataset/winequality-white.csv',sep=';')
df.head(3)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6


### Data Cleaning

#### Missing Value Handling: Identify and deal with missing values in the data.

In [10]:
null_values = df.isnull()
null_values.sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [11]:
df.dropna(inplace=True)

#### Duplicate Data Handling: Identify and deal with duplicate values in the data.

In [12]:
print(f'去除重复元素前df的形状: {df.shape}')
df.drop_duplicates(inplace=True)
print(f'去除重复元素后df的形状: {df.shape}')

去除重复元素前df的形状: (4898, 12)
去除重复元素后df的形状: (3961, 12)


### Data Integration

#### Combine data with the same attributes from different sources. For this assignment, calculate the “total acidity,” which is the sum of “fixed acidity” and “volatile acidity,” and add it as a new column to the dataset.

In [13]:
df['total acidity'] = df['fixed acidity'] + df['volatile acidity']
df.head(3)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,total acidity
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,7.27
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,6.60
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,8.38


### Data Transformation

#### Normalization: Normalize the “quality” data to the [0,1] range

In [14]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


# 初始化 MinMaxScaler
scaler = MinMaxScaler()
# Fit scaler 到数据并转换
df['quality_normalize'] = scaler.fit_transform(df[['quality']])
df.head(3)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,total acidity,quality_normalize
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,7.27,0.5
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,6.60,0.5
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,8.38,0.5


#### Discretization: Discretize the continuous attribute “fixed acidity” into three levels: “low,” “medium,” and “high.”

In [15]:
import pandas as pd

# 假设 df 是你的 DataFrame 并且 'fixed acidity' 是其中的一个连续属性列
# 使用 qcut 自动定义阈值，并将 'fixed acidity' 离散化为三个等级
df['fixed acidity level'] = pd.qcut(df['fixed acidity'], 3, labels=['low', 'medium', 'high'])
df.head(3)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,total acidity,quality_normalize,fixed acidity level
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,7.27,0.5,medium
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,6.60,0.5,low
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,8.38,0.5,high


### Data Reduction

#### Feature Selection: Use Analysis of Variance (ANOVA) to select the top three features that have the most significant impact on the quality rating of wine

In [16]:
from scipy.stats import f_oneway
# 创建一个空列表来存储F统计量和对应的p值
f_values = []
p_values = []
# # Separate features and target variable
X = df.drop(['quality','fixed acidity level','quality_normalize'], axis=1)  # feature columns
y = df['quality']  # target column
# 对每个特征进行ANOVA分析
for column in X.columns:
    # 使用f_oneway计算F统计量和p值
    f_statistic, p_value = f_oneway(X[column], y)
    f_values.append(f_statistic)
    p_values.append(p_value)

# 将结果转换为DataFrame
anova_results = pd.DataFrame({'Feature': X.columns, 'F Value': f_values, 'P Value': p_values})

# 按F值降序排序
anova_results.sort_values(by='F Value', ascending=False, inplace=True)

# 输出排名前三的特征
top_three_features = anova_results.head(3)
print("Top three features with the most significant impact on wine quality:")
print(top_three_features)

Top three features with the most significant impact on wine quality:
            Feature        F Value  P Value
4         chlorides  168367.769414      0.0
1  volatile acidity  153080.881913      0.0
2       citric acid  149342.977363      0.0
